In [1]:
from utils import *
from data import load_data

number_customer = 100
number_truck = 10
number_drone = 15
problem = load_data(r"data\100customers\r110.txt", number_customer, number_truck, number_drone)
for customer in problem.customer_list:
    print(customer)

Customer(x=35, y=35, demand=0, time=0.0, service=0.0)
Customer(x=41, y=49, demand=10, time=1.625, service=0.008333333333333333)
Customer(x=35, y=17, demand=7, time=0.25, service=0.008333333333333333)
Customer(x=55, y=45, demand=13, time=1.325, service=0.008333333333333333)
Customer(x=55, y=20, demand=19, time=0.8875, service=0.008333333333333333)
Customer(x=15, y=30, demand=26, time=0.25, service=0.008333333333333333)
Customer(x=25, y=30, demand=3, time=0.675, service=0.008333333333333333)
Customer(x=20, y=50, demand=5, time=0.825, service=0.008333333333333333)
Customer(x=10, y=43, demand=9, time=0.7625, service=0.008333333333333333)
Customer(x=55, y=60, demand=16, time=0.6625, service=0.008333333333333333)
Customer(x=30, y=60, demand=16, time=1.2625, service=0.008333333333333333)
Customer(x=20, y=65, demand=12, time=0.4125, service=0.008333333333333333)
Customer(x=50, y=35, demand=19, time=0.475, service=0.008333333333333333)
Customer(x=30, y=25, demand=23, time=0.875, service=0.00833

In [2]:
# chromosome = [[85, 101, 7, 23, 102, 68, 24, 21, 34, 38, 45, 4, 80, 10, 103, 65, 94, 104, 78, 36, 35, 57, 3, 105, 46, 67, 2, 49, 106, 99, 1, 30, 25, 39, 42, 72, 29, 107, 26, 37, 50, 69, 59, 44, 71, 28, 16, 48, 108, 86, 9, 89, 109, 40, 22, 13, 51, 58, 70, 66, 32, 14, 47, 60, 110, 77, 17, 73, 100, 111, 64, 12, 61, 97, 52, 112, 43, 113, 8, 114, 95, 87, 81, 62, 98, 75, 54, 33, 31, 90, 84, 92, 115, 19, 5, 116, 20, 117, 93, 63, 53, 91, 118, 27, 6, 82, 74, 15, 119, 11, 96, 88, 120, 18, 121, 55, 76, 83, 41, 79, 122, 56, 123], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
# truck_routes, drone_routes = extract_routes(chromosome, problem)
# truck_solutions, drone_solutions = find_solution(truck_routes, drone_routes, problem)

# for drone_sol in drone_solutions:
#     print(problem.check_capacity_drone_constraint(drone_sol))
#     for drone_trip in drone_sol.trip_list:
#         print(problem.check_energy_drone(drone_trip))

In [3]:
processing_number = 1
from moo_algorithm.nsga_ii import run_nsga_ii
# from utils import crossover_PMX, mutation_flip, init_random
pro_drone = 0.7
pop_size = 100
max_gen = 100
crossover_rate = 0.8
mutation_rate = 0.7
indi_list = []
for i in range(pop_size):
    indi = init_random(problem, pro_drone)
    indi_list.append(indi)
result_nsga_ii = run_nsga_ii(processing_number, problem, indi_list, pop_size, max_gen, crossover_PMX, mutation_flip, crossover_rate, mutation_rate, cal_fitness)

NSGA-II Done:  0 67789129856.142624
NSGA-II Done:  0 70001696221.18646
NSGA-II Done:  1 71622968750.17035
NSGA-II Done:  2 71676911550.13283
NSGA-II Done:  3 72361772564.09221
NSGA-II Done:  4 73681529950.34097
NSGA-II Done:  5 74103097554.02513
NSGA-II Done:  6 76143129326.64604
NSGA-II Done:  7 76164680808.10857
NSGA-II Done:  8 76464415010.46683
NSGA-II Done:  9 76481821804.23718


KeyboardInterrupt: 

In [ ]:
from moo_algorithm.pfg_moea import run_pfgmoea
GK = 5
sigma = 0.1

result_pfgmoea = run_pfgmoea(processing_number, problem, indi_list, pop_size, max_gen, GK, sigma, crossover_PMX, mutation_flip, 
            crossover_rate, mutation_rate, cal_fitness)

PFG MOEA
Generation 0:  0.0
Generation 1:  99128727.97902063
Generation 2:  99128727.97902063
Generation 3:  99128727.97902063
Generation 4:  1120939229.9879274
Generation 5:  1120939229.9879274
Generation 6:  1184933540.722226


KeyboardInterrupt: 